In [ ]:
#| default_exp framework_utils.fastai
#| default_cls_lvl 3

In [ ]:
#| hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# framework_utils.fastai

> As Jeremy says in his notebook, "Iteration speed is critical, so we need to quickly be able to try different data processing and trainer parameters" ... so lets create some helper methods to help us iterate more quickly.

In [ ]:
#| export
import os, gc, time

from fastai.callback.all import *
from fastai.data.block import ColReader, ColSplitter, DataBlock, IndexSplitter, RegressionBlock
from fastai.imports import *
from fastai.layers import SigmoidRange
from fastai.learner import *
from fastai.losses import MSELossFlat, L1LossFlat
from fastai.optimizer import Adam
from fastai.metrics import *
from fastai.torch_core import *
from fastai.torch_imports import *
from transformers import AutoConfig, AutoModelForMaskedLM, AutoModelForSequenceClassification, logging

from torch.nn import SmoothL1Loss

from blurr.callbacks import GradientCheckpointing
from blurr.text.data.core import TextBlock
from blurr.text.modeling.core import BaseModelWrapper, BaseModelCallback, blurr_splitter
from blurr.text.utils import get_hf_objects
from blurr.utils import PreCalculatedMSELoss, set_seed

from kaggle_comp import utils

In [ ]:
#| hide_input
import pdb
from fastcore.test import *


In [ ]:
#| hide
#| cuda
torch.cuda.set_device(0)
print(f"Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}")


Using GPU #0: NVIDIA GeForce RTX 3090


## Hugging Face `transformers` objects

In [ ]:
#| export
def get_comp_hf_objects(
    model_checkpoint,
    model_cls=AutoModelForSequenceClassification,
    config_kwargs={"num_labels": 6},
    tokenizer_kwargs={},
    model_kwargs={},
    new_special_tokens=None,
):

    # need to create configuration object separately because we may be adding new attributes (e.g., cls_dropout)
    config = AutoConfig.from_pretrained(model_checkpoint)
    config.update(config_kwargs)

    hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(
        model_checkpoint,
        model_cls=model_cls,
        config=config,
        tokenizer_kwargs=tokenizer_kwargs,
        model_kwargs=model_kwargs,
    )

    if new_special_tokens:
        # After adding the new tokens, we need to resize the embedding matrix in the model and initialize the weights
        hf_tokenizer.add_special_tokens({"additional_special_tokens": new_special_tokens})
        hf_model.resize_token_embeddings(len(hf_tokenizer))

        with torch.no_grad():
            emb_size = hf_model.config.to_dict().get("embedding_size", hf_model.config.hidden_size)
            hf_model.get_input_embeddings().weight[-len(hf_tokenizer), :] = torch.zeros([emb_size])

    return hf_arch, hf_config, hf_tokenizer, hf_model


## `DataLoaders`

In [ ]:
#| export
# NOTE: This is needed for the older models (since renamed to just "build_inputs")
def build_inputs_with_sep_and_section_toks(example, sep=" - ", lower_case=False, col="section_tok"):
    if lower_case:
        return f'{example[col].lower()}{sep}{example["context"].lower()}{sep}{example["anchor"].lower()}{sep}{example["target"].lower()}'
    else:
        return f'{example[col]}{sep}{example["context"]}{sep}{example["anchor"]}{sep}{example["target"]}'


In [ ]:
#| export
def build_inputs(
    example,
    sep=" [s] ",
    lower_case=True,
):

    full_text = example["full_text"].lower() if lower_case else example["full_text"]

    inp = full_text
    #inp = f"{context}{sep}{anchor}{sep}{target}"

    return inp


In [ ]:
#| export
def build_inputs_like_pretrained(example, sep=" [s] "):

    #return f"{section}{sep}{context}{sep}{anchor}", f"{target}"
    return example["full_text"]


In [ ]:
#| export
def get_dls(
    df,
    hf_arch,
    hf_config,
    hf_tokenizer,
    hf_model,
    get_x_func,
    val_idxs_or_fold,
    max_length=128,
    include_labels=True,
    tok_kwargs={},
    batch_size=4,
    seed=utils.default_seed,
    augment="none",
    sep=" [s] ",
):
    # define validation set
    if isinstance(val_idxs_or_fold, int):
        df["is_valid"] = df["k_fold"] == val_idxs_or_fold
        splitter = ColSplitter()
    else:
        splitter = IndexSplitter(val_idxs_or_fold)

    set_seed(seed)
    blocks = (
        TextBlock(hf_arch, hf_config, hf_tokenizer, hf_model, max_length=max_length, include_labels=include_labels, tok_kwargs=tok_kwargs),
        RegressionBlock(n_out = 6),
    )

    # move this to arg?
    target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']

    if augment == "none":
        dblock = DataBlock(blocks=blocks, get_x=ColReader('full_text'), get_y=ColReader(target_cols), splitter=splitter)
    elif augment == "rev":
        print("Using augmentation: rev")
        print("")
        dblock = DataBlock(
            blocks=blocks,
            get_x=ColReader('full_text'),
            get_y=ColReader(target_cols),
            splitter=splitter,
            item_tfms=utils.AugmentationTransform(utils.aug, phrase_fct=utils.rev_phrase, sep=sep),
        )

    set_seed(seed)
    return dblock.dataloaders(df, bs=batch_size)


## Models

In [ ]:
#| export
class YrangeBaseModelWrapper(BaseModelWrapper):
    def __init__(self, hf_config, hf_model, y_range=(1.0, 5.0), hf_model_kwargs={}):
        super().__init__(
            hf_model=hf_model,
            output_hidden_states=True,
            hf_model_kwargs=hf_model_kwargs,
        )
        store_attr()

        if y_range is not None:
            self.y_range = SigmoidRange(*y_range)

    def forward(self, x):
        res = super().forward(x)

        if self.y_range:
            return self.y_range(res.logits)

        return res.logits


In [ ]:
#| export
def yrange_blurr_splitter(m: Module):
    base_param_groups = blurr_splitter(m)

    added_groups = L([m for m_name, m in list(m.named_children()) if m_name != "hf_model"])
    base_param_groups[-1] += added_groups.map(params).filter(lambda el: len(el) > 0)

    return base_param_groups


In [ ]:
#| export
class CustomHeadModelWrapper(BaseModelWrapper):
    def __init__(self, hf_config, hf_model, p=0.0, y_range=None, layer_norm_eps=1e-5, hf_model_kwargs={}):
        super().__init__(
            hf_model=hf_model,
            output_hidden_states=True,
            hf_model_kwargs=hf_model_kwargs,
        )
        store_attr()

        self.layer_norm1 = nn.LayerNorm(self.hf_config.hidden_size, eps=self.layer_norm_eps)
        self.fc_dropout = nn.Dropout(p)
        self.fc = nn.Linear(self.hf_config.hidden_size, 1)

        if y_range is not None:
            self.y_range = SigmoidRange(*y_range)

    def forward(self, inputs):
        outputs = super().forward(inputs)
        last_hidden_state = outputs[0]

        input_mask_expanded = inputs["attention_mask"].unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)

        out = sum_embeddings / sum_mask
        out = self.layer_norm1(out)
        logits = self.fc(self.fc_dropout(out))

        if self.y_range:
            return self.y_range(logits)

        return logits


In [ ]:
#| export
def custom_head_blurr_splitter(m: Module):
    base_param_groups = blurr_splitter(m)

    added_groups = L([m for m_name, m in list(m.named_children()) if m_name != "hf_model"])
    added_param_groups = added_groups.map(params).filter(lambda el: len(el) > 0)

    return base_param_groups + added_param_groups


## `Learner`

In [ ]:
#| export
def get_learner(
    dls,
    hf_model,
    custom_base_model=None,
    adam_beta1=0.9,
    adam_beta2=0.98,
    adam_eps=1e-6,
    weight_decay=0.01,
    splitter=blurr_splitter,
    use_fp16=True,
    seed=utils.default_seed,
):
    # configure the model, loss function and learner
    if not custom_base_model:
        model = BaseModelWrapper(hf_model)
        loss_func = utils.MCRMSELoss()
        learn_cbs = [BaseModelCallback]
    else:
        model = custom_base_model
        loss_func = SmoothL1Loss(reduction = "mean")
        learn_cbs = []

    set_seed(seed)

    learn = Learner(
        dls,
        model,
        opt_func=partial(Adam, mom=adam_beta1, sqr_mom=adam_beta2, eps=adam_eps, wd=weight_decay),
        moms=(adam_beta1, adam_beta1 - 0.10, adam_beta1),
        cbs=learn_cbs,
        splitter=splitter,
    )

    learn.loss_func = loss_func
    learn.metrics = [utils.MCRMSE()]
    learn.create_opt()
    learn.freeze()

    if use_fp16:
        learn = learn.to_fp16()

    return learn


In [ ]:
#| export
class FastaiCompTrainer(utils.CompTrainer):
    def __init__(self, train_config, model_name, model_output_path="models", log_output_path="logs", **kwargs):
        super().__init__(
            train_config=train_config, model_name=model_name, model_output_path=model_output_path, log_output_path=log_output_path, **kwargs
        )

    def train(
        self,
        CFG,
        data,
        experiment_name: str,
        n_fold: int,
        run_id: str,
        grid_id: int,
        seed=None,
        verbose: bool = True,
    ):
        # timing
        start = time.time()
        
        if seed is None:
            seed = CFG.random_seed

        if verbose:
            print(f"Experiment: {experiment_name}")
            print(f"Run: {run_id} | Grid ID: {grid_id} | Fold: {n_fold}")
            print(f"Training config: f{self.train_config}")

        # from the experiment's training config if not in CFG
        # ------
        model_checkpoint = self.get_value_for("model_checkpoint", CFG, None)
        hf_model_cls = self.get_value_for("hf_model_cls", CFG, AutoModelForSequenceClassification)
        using_pretrained = self.get_value_for("using_pretrained", CFG, False)
        hf_config_kwargs = self.get_value_for("hf_config_kwargs", CFG, {"num_labels": 6})
        hf_tokenizer_kwargs = self.get_value_for("hf_tokenizer_kwargs", CFG, {})

        # datablock/dataloaders
        max_length = self.get_value_for("max_length", CFG, 256)
        include_labels = self.get_value_for("include_labels", CFG, True)
        tok_kwargs = self.get_value_for("tok_kwargs", CFG, {})
        batch_size = self.get_value_for("batch_size", CFG, 4)

        # learner
        adam_beta1 = self.get_value_for("adam_beta1", CFG, 0.9)
        adam_beta2 = self.get_value_for("adam_beta2", CFG, 0.999)
        adam_eps = self.get_value_for("adam_eps", CFG, 1e-6)
        weight_decay = self.get_value_for("weight_decay", CFG, 0.01)
        max_grad_norm = self.get_value_for("max_grad_norm", CFG, None)
        splitter = self.get_value_for("splitter", CFG, blurr_splitter)
        save_best_model = self.get_value_for("save_best_model", CFG, True)
        use_fp16 = self.get_value_for("use_fp16", CFG, True)
        custom_base_model_cls = self.get_value_for("custom_base_model_cls", CFG, None)
        custom_base_model_kwargs = self.get_value_for("custom_base_model_kwargs", CFG, {})

        # training
        n_frozen_epochs = self.get_value_for("n_frozen_epochs", CFG, 0)
        frozen_lr = self.get_value_for("frozen_lr", CFG, 1e-2)
        n_unfrozen_epochs = self.get_value_for("n_unfrozen_epochs", CFG, 5)
        unfrozen_lrs = self.get_value_for("unfrozen_lrs", CFG, [1e-5, 1e-2])
        # ------

        # move this to arg?
        target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']

        df = data.copy()
        #df["section_tok"] = df.section.apply(lambda v: f"[{v}]")

        # step 1: hf objects
        if verbose:
            print("Building HF objects ...")

        #if not using_pretrained:
        #    pass
        #    new_special_tokens = list(df.section_tok.unique())
        #    new_special_tokens.append("[s]")
        #else:
        new_special_tokens = None

        set_seed(seed)
        hf_arch, hf_config, hf_tokenizer, hf_model = get_comp_hf_objects(
            model_checkpoint,
            model_cls=hf_model_cls,
            config_kwargs=hf_config_kwargs,
            tokenizer_kwargs=hf_tokenizer_kwargs,
            new_special_tokens=new_special_tokens,
        )

        # step 2: dataloaders
        if verbose:
            print("Building DataLoaders ...")

        if not using_pretrained:
            get_x_func = partial(build_inputs)
        else:
            get_x_func = partial(build_inputs_like_pretrained)

        dls = get_dls(
            df,
            hf_arch,
            hf_config,
            hf_tokenizer,
            hf_model,
            get_x_func=get_x_func,
            val_idxs_or_fold=n_fold,
            max_length=max_length,
            include_labels=include_labels,
            tok_kwargs=tok_kwargs,
            batch_size=batch_size,
            seed=seed,
            augment=CFG.augment,
        )

        b = dls.one_batch()
        print(f"b0 shape: {b[0]['input_ids'].shape}; b1 shape: {b[1].shape}")

        # step 3: train
        if verbose:
            print("Building Learner ...")

        base_model_wrapper = None
        if custom_base_model_cls:
            base_model_wrapper = custom_base_model_cls(hf_config=hf_config, hf_model=hf_model, **custom_base_model_kwargs)

        self.learn = get_learner(
            dls,
            hf_model,
            custom_base_model=base_model_wrapper,
            adam_beta1=adam_beta1,
            adam_beta2=adam_beta2,
            adam_eps=adam_eps,
            weight_decay=weight_decay,
            splitter=splitter,
            use_fp16=use_fp16,
            seed=seed,
        )

        fit_cbs = []
        if max_grad_norm:
            fit_cbs.append(GradientClip(max_norm=max_grad_norm))

        if save_best_model:
            fit_cbs.append(
                SaveModelCallback(
                    monitor="mcrmse",
                    comp=np.less,
                    fname=f"{self.model_name}_best_mcrsme",
                    reset_on_fit=False,
                )
            )

        if n_frozen_epochs > 0:
            set_seed(seed)
            self.learn.fit_one_cycle(n_frozen_epochs, lr_max=frozen_lr, cbs=fit_cbs)

        if n_unfrozen_epochs > 0:
            self.learn.unfreeze()
            set_seed(seed)
            self.learn.fit_one_cycle(n_unfrozen_epochs, lr_max=slice(*unfrozen_lrs), cbs=fit_cbs)

        # step 4: log run details and performance on OOF data
        log_df = pd.DataFrame(self.learn.recorder.values, columns=self.learn.recorder.metric_names[1:-1])
        log_df["epoch"] = log_df.index
        log_df["fold"] = n_fold
        log_df["experiment"] = experiment_name
        log_df["run_id"] = run_id
        log_df["grid_id"] = grid_id
        log_df["subset"] = CFG.subset
        log_df["seed"] = seed
        log_df["batch_size"] = batch_size
        log_df["max_length"] = max_length
        log_df["n_frozen_epochs"] = n_frozen_epochs
        log_df["frozen_lr"] = frozen_lr
        log_df["n_unfrozen_epochs"] = n_unfrozen_epochs
        log_df["unfrozen_lrs"] = str(unfrozen_lrs)[1:-1]
        log_df["adam_beta1"] = adam_beta1
        log_df["adam_beta2"] = adam_beta2
        log_df["adam_beta2"] = adam_beta2
        log_df["adam_eps"] = adam_eps
        log_df["weight_decay"] = weight_decay
        log_df["max_grad_norm"] = max_grad_norm
        log_df["save_best_model"] = save_best_model
        log_df["use_fp16"] = use_fp16
        #log_df["strat_feat"] = CFG.strat_feat
        log_df["preprocess"] = CFG.preprocess
        log_df["postprocess"] = CFG.postprocess
        log_df["augment"] = CFG.augment

        # oof df
        preds, targs = self.learn.get_preds()
        preds = pd.DataFrame(preds)
        preds.columns = ["pred_" + x for x in target_cols]
        targs = pd.DataFrame(targs)
        targs.columns = ["targ_" + x for x in target_cols]
        oof_df = pd.concat([preds, targs], axis = 1)
        oof_df["run_id"] = run_id
        oof_df["grid_id"] = grid_id

        # time keeping
        log_df["time"] = time.time() - start

        # save model
        if verbose:
            print("--- saving model ---")

        self.learn.metrics = None
        self.learn.export(self.model_output_path / f"{self.model_name}.pkl")

        # clean up
        del self.learn, dls
        torch.cuda.empty_cache()
        gc.collect()

        return log_df, oof_df

    def predict(self, model_name, data):
        pass


## Export -

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()